# Warhammer 40K Miniature Detector - YOLO11x Training

Train the **best available YOLO model** (YOLO11x - 57M parameters) on your annotated dataset.

## Setup
1. **Runtime > Change runtime type > T4 GPU** (free tier)
2. Upload `yolo_dataset.zip` when prompted
3. Run all cells
4. Training takes ~2 hours on T4 GPU

## Expected Results
- Current model (YOLOv8n): 63.2% mAP50
- Target (YOLO11x): 75-85% mAP50

In [ ]:
# Install YOLOv8/YOLO11
!pip install ultralytics -q
print("Ultralytics installed!")

In [ ]:
# Check GPU
import torch
print(f"GPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("WARNING: No GPU detected! Go to Runtime > Change runtime type > T4 GPU")

In [ ]:
# Upload your dataset
from google.colab import files
print("Upload yolo_dataset.zip (49MB)")
print("You can find this at: photoanalyzer/backend/yolo_dataset.zip")
uploaded = files.upload()

In [ ]:
# Extract dataset
!unzip -q yolo_dataset.zip -d /content/
!ls /content/yolo_dataset/

import os
train_count = len(os.listdir('/content/yolo_dataset/images/train'))
val_count = len(os.listdir('/content/yolo_dataset/images/val'))
print(f"\nTrain images: {train_count}")
print(f"Val images: {val_count}")
print(f"Total: {train_count + val_count}")

In [ ]:
# Update data.yaml path for Colab
yaml_content = '''# YOLO Dataset Configuration
path: /content/yolo_dataset
train: images/train
val: images/val

# Classes
nc: 8
names: ["adeptus_mechanicus", "chaos_space_marines", "custodes", "death_guard", "eldar", "genestealer_cult", "grey_knights", "imperial_guard"]
'''

with open('/content/yolo_dataset/data.yaml', 'w') as f:
    f.write(yaml_content)

print("data.yaml updated for Colab!")
!cat /content/yolo_dataset/data.yaml

## Training with YOLO11x

YOLO11x is the latest and most accurate YOLO model:
- 57M parameters (vs 3M for YOLOv8n)
- State-of-the-art accuracy
- ~2 hours training on T4 GPU

In [ ]:
from ultralytics import YOLO

# Load YOLO11x - the best model
model = YOLO('yolo11x.pt')
print("YOLO11x loaded!")

In [ ]:
# Train with optimal settings
results = model.train(
    data='/content/yolo_dataset/data.yaml',
    
    # Training duration
    epochs=100,
    patience=15,  # Early stopping
    
    # Resolution
    imgsz=640,
    
    # Batch size (adjust if out of memory)
    batch=16,
    
    # Augmentation
    degrees=15,
    translate=0.1,
    scale=0.5,
    shear=5,
    flipud=0.1,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.15,
    copy_paste=0.1,
    
    # Regularization
    dropout=0.1,
    
    # Output
    save=True,
    save_period=10,
    project='/content/runs',
    name='warhammer_yolo11x',
    exist_ok=True,
    verbose=True,
    plots=True,
)

In [ ]:
# View training results
from IPython.display import Image, display
display(Image('/content/runs/warhammer_yolo11x/results.png', width=800))

In [ ]:
# View confusion matrix
display(Image('/content/runs/warhammer_yolo11x/confusion_matrix.png', width=600))

In [ ]:
# Validate and show final metrics
best_model = YOLO('/content/runs/warhammer_yolo11x/weights/best.pt')
metrics = best_model.val(data='/content/yolo_dataset/data.yaml')

print("\n" + "="*50)
print("FINAL MODEL METRICS")
print("="*50)
print(f"mAP50:     {metrics.box.map50:.1%}")
print(f"mAP50-95:  {metrics.box.map:.1%}")
print(f"Precision: {metrics.box.mp:.1%}")
print(f"Recall:    {metrics.box.mr:.1%}")
print("="*50)

In [ ]:
# Test on validation images
import os
val_images = os.listdir('/content/yolo_dataset/images/val/')[:6]

for img in val_images:
    results = best_model.predict(
        f'/content/yolo_dataset/images/val/{img}',
        save=True,
        project='/content/predictions',
        conf=0.25
    )
    print(f"Processed {img}")

In [ ]:
# Show predictions
import glob
pred_images = sorted(glob.glob('/content/predictions/predict/*.jpg'))[:6]
for img in pred_images:
    print(f"\n{os.path.basename(img)}")
    display(Image(img, width=500))

## Download Trained Model

Download the best model weights to use in your application.

In [ ]:
# Download the trained model
from google.colab import files

print("Downloading best.pt (~110MB)...")
files.download('/content/runs/warhammer_yolo11x/weights/best.pt')

In [ ]:
# Optional: Download all training artifacts
!zip -r /content/training_results.zip /content/runs/warhammer_yolo11x/
files.download('/content/training_results.zip')

## Using the Model

After downloading, use the model like this:

```python
from ultralytics import YOLO

# Load your trained model
model = YOLO('best.pt')

# Run inference
results = model.predict('image.jpg', conf=0.25)

# Get detections
for r in results:
    for box in r.boxes:
        cls = int(box.cls[0])
        conf = float(box.conf[0])
        name = model.names[cls]
        print(f"{name}: {conf:.1%}")
```